# Libraries

In [ ]:
library("tidyverse")
library("viridis")

# Data

Reading in the data

In [ ]:
impostor_raw <- read.csv("LLM_Battles/impostor_techniques_usage.csv")
crewmate_raw <- read.csv("LLM_Battles/crewmate_techniques_usage.csv")
tournament_raw <- read.csv("LLM_Battles/tournament_analysis_640.csv")

Droping the "Total.Techniques" column

In [ ]:
impostor_droped <- impostor_raw %>%
  select(-"Total.Techniques")

crewmate_droped <- crewmate_raw %>%
  select(-"Total.Techniques")

Changing format to long

In [ ]:
impostor_long <- impostor_droped %>%
  pivot_longer(
    cols = -Persuasion.Technique,
    names_to = "model",
    values_to = "value"
  )

crewmate_long <- crewmate_droped %>%
  pivot_longer(
    cols = -Persuasion.Technique,
    names_to = "model",
    values_to = "value"
  )

Combining datasets

In [ ]:
crewmate_long$role <- "crewmate"
impostor_long$role <- "impostor"

data <- bind_rows(crewmate_long, impostor_long)

Creating a size to name translation

In [ ]:
sizes <- unique(tournament_raw[c("Impostor", "ImpostorSize")]) %>%
  rename(
    model_tournament = Impostor,
    size = ImpostorSize
  )
sizes$model <- c(
  "llama.3.1.8b.instruct",
  "llama.3.1.405b.instruct",
  "gpt.4o.mini",
  "gpt.4o",
  "gemini.flash.1.5",
  "gemini.pro.1.5",
  "claude.3.5.sonnet",
  "claude.3.5.haiku"
)

Adding size label to the data

In [ ]:
data <- data %>% left_join(sizes %>% select(model, size), by = "model")

Averaging the crewmate values

In [ ]:
data <- data %>% mutate(value = if_else(role == "crewmate", value / 4, value))

# Plots

### Points

In [ ]:
ggplot(data, aes(y = Persuasion.Technique, x = value, color = model)) +
  geom_point(size = 3) + scale_x_sqrt() + facet_grid(size ~ role)

### Bar plot

In [ ]:
ggplot(data, aes(x = model, y = value, fill = as.factor(Persuasion.Technique))) +
  geom_bar(stat = "identity", position = "dodge", color = "#000000") +
  facet_grid(~role) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank()
  )

### Heatmap

In [ ]:
ggplot(data, aes(x = model, y = Persuasion.Technique, fill = value)) +
  geom_tile(color = "white") +
  scale_fill_gradient(low = "lightyellow", high = "orange") +
  geom_text(aes(label = as.integer(value)), size = 3) +
  labs(title = "Heatmap of Persuasion Values by Model",
       x = "Model",
       y = "Persuasion Technique",
       fill = "Value") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank()
  ) +
  facet_wrap(~role, ncol = 2)

In [ ]:
ggsave("final_plot.pdf", width = 10, height = 6)